## rubin_sim movingObjects ##

### Dealing with orbits ###

The movingObjects module is focused on generating simulated observations of moving objects in the LSST (or any other survey). Here we demonstrate the first step of this, generating ephemerides with PyOorb in the rubin_sim.movingObjects package, as this may be useful in other cases as well.

The module uses PyOorb (the python interface to openorb) to do the ephemeris generation, but the code is structured so that propagation and ephemeris generation could be handled by a different integrator. 

In [1]:
import numpy as np
import pandas as pd
import rubin_sim.moving_objects as mo

In [2]:
# Read in a test orbit file 
# Note that the format for the orbit file should match the expected format for the FORTRAN interface of OpenOrb -
# This means the standard .mpc, .s3m or .des files are readable. 
# Angles should be in degrees, dates in MJD (timescale is assumed to be TT)

orbits = mo.Orbits()
orbits.read_orbits('test.des')

In [3]:
# The orbit information is stored in a pandas dataframe
orbits.orbits

,obj_id,q,e,inc,Omega,argPeri,tPeri,H,epoch,g,sed_filename
0,S120000,2.726805,0.143390,6.376748,97.179890,293.230208,53771.523748,17.582,54466.0,0.15,C.dat
1,S120001,2.649788,0.056054,7.842717,171.580510,143.356388,54967.972509,20.514,54466.0,0.15,S.dat
2,S120002,2.823276,0.080222,11.639677,97.850771,331.899280,53606.683317,18.304,54466.0,0.15,S.dat


So: ephemeris generation.

In [5]:
# The ephemeris generation and propagation is handled by PyOrbEphemerides, which is separate from Orbits to 
# provide the opportunity to swap out ephemeris generation.

pyephs = mo.PyOrbEphemerides()
pyephs.set_orbits(orbits)

Note that within `pyephs` we keep a copy of the orbits .. but in a different format, appropriate for use with PyOorb.

In [6]:
# Generate ephemerides at a series of dates .. let's use every other day for 30 days from the original epoch

times = np.arange(orbits.orbits['epoch'][0], orbits.orbits['epoch'][0] + 30, 2)


# byObject = True means that the ephemerides will be grouped by object in the returned 3-d array ..
# if this is 'False' the ephemerides are grouped by time instead 
# basic == a minimal set of ephemeris values. full == the entire set of PyOorb ephemeris values.

ephs = pyephs.generate_ephemerides(times, time_scale='UTC', obscode='I11', 
                         by_object=True, 
                         eph_mode='nbody', eph_type='basic')

In [7]:
# Because we chose 'byObject', the first element contains ephemerides for the first object.
# (if we chose 'byObject=False', then first element contains ephemerides for all objects at the first time)
pd.DataFrame(ephs[0])

,time,ra,dec,dradt,ddecdt,phase,solarelon,helio_dist,geo_dist,magV,trueAnomaly,velocity
0,54466.0,182.578901,5.880676,0.104457,-0.003417,16.273603,99.738741,3.458398,3.153541,23.649852,133.382199,0.104513
1,54468.0,182.773102,5.877476,0.096169,0.000632,16.162639,101.584264,3.460424,3.126235,23.628547,133.673035,0.096171
2,54470.0,182.950500,5.882399,0.087682,0.004732,16.033772,103.447842,3.462440,3.099108,23.606593,133.963533,0.087809
3,54472.0,183.110714,5.895535,0.079005,0.008872,15.886645,105.329632,3.464447,3.072195,23.583997,134.253692,0.079501
4,54474.0,183.253390,5.916952,0.070152,0.013040,15.720928,107.229729,3.466443,3.045532,23.560773,134.543517,0.071354
5,54476.0,183.378203,5.946693,0.061141,0.017224,15.536337,109.148159,3.468429,3.019157,23.536933,134.833009,0.063521
6,54478.0,183.484865,5.984775,0.051992,0.021409,15.332633,111.084886,3.470406,2.993106,23.512495,135.122171,0.056228
7,54480.0,183.573130,6.031187,0.042724,0.025585,15.109629,113.039832,3.472373,2.967417,23.487475,135.411004,0.049799
8,54482.0,183.642789,6.085896,0.033355,0.029739,14.867186,115.012905,3.474329,2.942127,23.461894,135.699511,0.044688
9,54484.0,183.693659,6.148854,0.023897,0.033867,14.605203,117.004033,3.476275,2.917270,23.435769,135.987695,0.041449


In [8]:
# Another example with some different choices - 
# byObject = False gives all objects at the first time as the first element
# ephType = full gives a more complete (but slightly slower) set of ephemeris values
ephs = pyephs.generate_ephemerides(times, time_scale='UTC', obscode='I11', 
                                  by_object=False, 
                                  eph_mode='nbody', eph_type='full')
pd.DataFrame(ephs[0])

,time,ra,dec,dradt,ddecdt,phase,solarelon,helio_dist,geo_dist,magV,...,helio_y,helio_z,helio_dx,helio_dy,helio_dz,obs_helio_x,obs_helio_y,obs_helio_z,trueAnom,velocity
0,54466.0,182.578901,5.880676,0.104457,-0.003417,16.273603,99.736650,3.458398,3.153541,23.649852,...,0.967917,0.352544,-0.003400,-0.008147,0.000491,-0.16765,0.968894,-0.000042,133.382199,0.104513
1,54466.0,219.410575,-11.528761,0.330045,-0.069344,17.203783,59.226751,2.856409,3.231703,26.250452,...,-1.132464,0.207030,0.004547,-0.008935,0.001126,-0.16765,0.968894,-0.000042,248.731283,0.337251
2,54466.0,241.098229,-13.631772,0.318426,-0.063875,10.716174,38.606251,3.299593,4.010445,24.597699,...,-2.537566,0.489999,0.006733,-0.006015,-0.001205,-0.16765,0.968894,-0.000042,160.708320,0.324769


Propagation is also possible with PyOrbEphemerides + PyOorb. 

In [9]:
# Remember that pyephs is storing the orbits internally, in a format that is not necessarily the easiest to read.
# 'Orbits' copy (nice dataframe)
orbits.orbits

,obj_id,q,e,inc,Omega,argPeri,tPeri,H,epoch,g,sed_filename
0,S120000,2.726805,0.143390,6.376748,97.179890,293.230208,53771.523748,17.582,54466.0,0.15,C.dat
1,S120001,2.649788,0.056054,7.842717,171.580510,143.356388,54967.972509,20.514,54466.0,0.15,S.dat
2,S120002,2.823276,0.080222,11.639677,97.850771,331.899280,53606.683317,18.304,54466.0,0.15,S.dat


In [10]:
# pyephs copy - packaged numpy array
pyephs.oorb_elem

array([[1.00000000e+00, 2.72680464e+00, 1.43389847e-01, 1.11295242e-01,
        1.69610904e+00, 5.11783260e+00, 5.37715237e+04, 2.00000000e+00,
        5.44660000e+04, 3.00000000e+00, 1.75820000e+01, 1.50000000e-01],
       [2.00000000e+00, 2.64978782e+00, 5.60540098e-02, 1.36881230e-01,
        2.99464483e+00, 2.50204097e+00, 5.49679725e+04, 2.00000000e+00,
        5.44660000e+04, 3.00000000e+00, 2.05140000e+01, 1.50000000e-01],
       [3.00000000e+00, 2.82327567e+00, 8.02216088e-02, 2.03150687e-01,
        1.70781814e+00, 5.79273522e+00, 5.36066833e+04, 2.00000000e+00,
        5.44660000e+04, 3.00000000e+00, 1.83040000e+01, 1.50000000e-01]])

In [11]:
# Propagate the orbits forward 60 days.
pyephs.propagate_orbits(new_epoch=orbits.orbits['epoch'][0] + 60, eph_mode='nbody')

In [12]:
# Updated orbits
pyephs.oorb_elem

array([[1.00000000e+00, 2.72641441e+00, 1.43444271e-01, 6.37674117e+00,
        9.71797234e+01, 2.93203622e+02, 3.08087723e+06, 2.00000000e+00,
        5.45260000e+04, 3.00000000e+00, 1.75820000e+01, 1.50000000e-01],
       [2.00000000e+00, 2.64979694e+00, 5.60887866e-02, 7.84261202e+00,
        1.71579899e+02, 1.43309602e+02, 3.14942011e+06, 2.00000000e+00,
        5.45260000e+04, 3.00000000e+00, 2.05140000e+01, 1.50000000e-01],
       [3.00000000e+00, 2.82456097e+00, 7.99963446e-02, 1.16396922e+01,
        9.78506872e+01, 3.31925768e+02, 3.07142796e+06, 2.00000000e+00,
        5.45260000e+04, 3.00000000e+00, 1.83040000e+01, 1.50000000e-01]])

In [13]:
# And we can pull these back out to a nice dataframe like this: 
new_orbits = pyephs.convert_from_oorb_elem()

In [14]:
new_orbits

,q,e,inc,Omega,argPeri,tPeri,epoch,H,g
0,2.726414,0.143444,365.360356,5567.988002,16799.330078,3.080877e+06,54526.0,17.582,0.15
1,2.649797,0.056089,449.348569,9830.804043,8211.035368,3.149420e+06,54526.0,20.514,0.15
2,2.824561,0.079996,666.905240,5606.431397,19017.945614,3.071428e+06,54526.0,18.304,0.15


In [15]:
# Note that because pyOorb doesn't use all of the contents of our original Orbits dataframe (the SED, for example)
# that some of those quantities are missing here -- 
orbits.orbits

,obj_id,q,e,inc,Omega,argPeri,tPeri,H,epoch,g,sed_filename
0,S120000,2.726805,0.143390,6.376748,97.179890,293.230208,53771.523748,17.582,54466.0,0.15,C.dat
1,S120001,2.649788,0.056054,7.842717,171.580510,143.356388,54967.972509,20.514,54466.0,0.15,S.dat
2,S120002,2.823276,0.080222,11.639677,97.850771,331.899280,53606.683317,18.304,54466.0,0.15,S.dat


In [16]:
# But we can update the original Orbits to include them:
orbits.update_orbits(new_orbits)
orbits.orbits

,obj_id,q,e,inc,Omega,argPeri,tPeri,epoch,H,g,sed_filename
0,S120000,2.726414,0.143444,365.360356,5567.988002,16799.330078,3.080877e+06,54526.0,17.582,0.15,C.dat
1,S120001,2.649797,0.056089,449.348569,9830.804043,8211.035368,3.149420e+06,54526.0,20.514,0.15,S.dat
2,S120002,2.824561,0.079996,666.905240,5606.431397,19017.945614,3.071428e+06,54526.0,18.304,0.15,S.dat
